<a href="https://colab.research.google.com/github/omkarpat/BotBuilder-Location/blob/master/Baseline_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Colab settings/mount
from google.colab import drive
drive.mount('/content/gdrive')
%cd gdrive/My\ Drive/CSE\ 240/Project/Data/spectrogram_images/

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/My Drive/CSE 240/Project/Data/spectrogram_images


In [2]:
import os
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

#%tensorflow_version 2.x
#import tensorflow
#print(tensorflow.__version__)
from keras import optimizers
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Conv2D, Dense, Flatten, MaxPooling2D, SeparableConv2D, MaxPooling3D, Conv3D, DepthwiseConv2D
from keras.models import model_from_json

Using TensorFlow backend.


In [3]:
base_path = "."
concentration_data_array = None
relaxed_data_array = None
#relaxed_data_array = np.load("relaxed_array_updated_spectrograms_final.npy")
for filename in os.listdir(base_path):
  if "concentration_array_updated_spectrograms" in filename and "final" not in filename:
    print(filename)
    concentration_data_array = np.concatenate((concentration_data_array, np.load(filename))) if concentration_data_array is not None else np.load(filename)
  if "relaxed_array_updated_spectrograms" in filename:
    print(filename)
    relaxed_data_array = np.concatenate((relaxed_data_array, np.load(filename))) if relaxed_data_array is not None else np.load(filename)

concentration_array_updated_spectrograms_1.npy
concentration_array_updated_spectrograms_2.npy
concentration_array_updated_spectrograms_3.npy
concentration_array_updated_spectrograms_4.npy
concentration_array_updated_spectrograms_5.npy
concentration_array_updated_spectrograms_6.npy
relaxed_array_updated_spectrograms_1.npy
relaxed_array_updated_spectrograms_2.npy
relaxed_array_updated_spectrograms_3.npy
relaxed_array_updated_spectrograms_4.npy
relaxed_array_updated_spectrograms_5.npy
relaxed_array_updated_spectrograms_final.npy


In [4]:
print(concentration_data_array.shape, relaxed_data_array.shape)

(2160, 29, 43, 42) (2124, 29, 43, 42)


In [0]:
X = np.concatenate((relaxed_data_array, concentration_data_array))
relaxed_data_array, concentration_data_array = None, None

In [6]:
Y = np.concatenate((np.zeros((2160,1)),np.ones((2124,1))))
print(X.shape, Y.shape)

(4284, 29, 43, 42) (4284, 1)


In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.15, random_state=42)
X, Y = None, None
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.15, random_state=42)
print(X_train.shape, X_test.shape, X_valid.shape, y_train.shape, y_test.shape, y_valid.shape)

(3094, 29, 43, 42) (643, 29, 43, 42) (547, 29, 43, 42) (3094, 1) (643, 1) (547, 1)


In [8]:
model2 = Sequential()
model2.add(Conv2D(64, kernel_size=5, strides=(1,1), padding='valid', activation='relu', input_shape=(29, 43, 42)))
model2.add(Conv2D(128, kernel_size=3, activation='relu'))
model2.add(MaxPooling2D(pool_size=(2, 2)))
model2.add(Conv2D(64, kernel_size=5, strides=(1,1), padding='valid', activation='relu'))
model2.add(Conv2D(128, kernel_size=3, activation='relu'))
model2.add(MaxPooling2D(pool_size=(2, 2)))
model2.add(Flatten())
model2.add(Dense(100, activation='relu'))
model2.add(Dense(50, activation='relu'))
model2.add(Dense(1, activation='sigmoid'))
sgd = optimizers.SGD(lr=0.000001, momentum=0.7, decay=1e-6)
model2.compile(optimizer=sgd, loss='binary_crossentropy', metrics=['accuracy'])
# checkpoint
filepath="model2-weights-improvement-{epoch:02d}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]
model2.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs=50, batch_size=32, callbacks=callbacks_list, shuffle=True)
metrics = model2.evaluate(X_test,y_test)
print(metrics)
model2_json = model2.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model2_json)
# serialize weights to HDF5
model2.save_weights("model.h5")
print("Saved model 2 to disk")







Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Train on 3094 samples, validate on 547 samples
Epoch 1/50







KeyboardInterrupt: ignored

In [9]:
# load json and create model
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")

# evaluate loaded model on test data
loaded_model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
score = loaded_model.evaluate(X_test,y_test, verbose=0)
print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))

Loaded model from disk
acc: 54.28%


In [10]:
score = loaded_model.evaluate(X_train, y_train, verbose=0)
print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))

acc: 62.31%
